#### Bayesian inference and Data assimilation SS2023

# Exercise 11 sample sample solution

#### Jin W. Kim (jin.won.kim@uni-potsdam.de)

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Problem 1

#### 1.1 Prediction step
The distribution of $X_1$ is given by

$$
X_1 = \frac{1}{2}X_0 + 1 + \Xi_0
$$

Since $X_0\sim N(-1,2)$ and $\Xi_0 \sim N(0,1)$, $X_1$ is again Gaussian with the mean and the variance given by

$$
X_1 \sim N\big(-\frac{1}{2} + 1, \frac{2}{4} + 0 + 1\big) = N\big(\frac{1}{2}, \frac{3}{2}\big)
$$

#### 1.2 Filtering step (preliminary, not necessary for marking)
We first derive the conditional distribution briefly. suppose $\pi_{X} \sim N(m,\sigma_x)$ and $Y = hX + N(0,\sigma_y)$, then

$$
\begin{aligned}
\pi_{X|Y}(x|y) &\propto \pi_{Y|X}(y|x) \pi_X(x) \\
&\propto \exp\Big(-\frac{1}{2\sigma_y}(y-hx)^2-\frac{1}{2\sigma_x}(x-m)^2\Big) \\
&\propto\exp\Big(-\frac{1}{2}\big(\frac{h^2}{\sigma_y}x^2 - \frac{2yh}{\sigma_y}x + \frac{y^2}{\sigma_y}+\frac{1}{\sigma_x}x^2-\frac{2m}{\sigma_x}x + \frac{m^2}{\sigma_x}\big)\Big)
\end{aligned}
$$

Define

$$
\sigma_{x|y}:=1/(h^2/\sigma_y+1/\sigma_x) = \frac{\sigma_x\sigma_y}{h^2\sigma_x+\sigma_y}
$$

Then

$$
\pi_{X|Y}(x|y) \propto\exp\Big(-\frac{1}{2\sigma_{x|y}}\big(x^2 - 2(\frac{\sigma_{x|y}}{\sigma_y}yh+\frac{\sigma_{x|y}}{\sigma_x}m)x \big)\Big)
$$

This is Gaussian with

$$
\pi_{X|Y}(x|y) \sim N\Big(\frac{h\sigma_{x|y}}{\sigma_y}y+\frac{\sigma_{x|y}}{\sigma_x}m, \sigma_{x|y}\Big)
$$

It is often expressed using the Kalman gain

$$
K := \frac{h\sigma_{x|y}}{\sigma_y} = \frac{h\sigma_x}{h^2\sigma_x+\sigma_y}
$$

Then the mean $Ky+\frac{\sigma_{x|y}}{\sigma_x}m = m + K(y-hm)$, and therefore

$$
\pi_{X|Y}(x|y) \sim N\big(m + K(y-hm), \frac{1}{h}K\sigma_y\big)
$$


#### 1.2 Filtering step
The noise variance is 2 and $h=1$, and the mean is as given in step 1. Therefore, we have

$$
K = \frac{\sigma_x}{\sigma_x+\sigma_y} = \frac{3/2}{3/2+2} = \frac{3}{7}
$$

Therefore the conditional mean

$$
\mathbb{E}\big[X_1 | Y_1 = 2\big] = \frac{1}{2} + \frac{3}{7}\big(2-\frac{1}{2}\big) = \frac{8}{7}
$$

and the conditional variance

$$
\text{Var}\big[X_1 | Y_1=2\big] = 2\cdot \frac{3}{7} = \frac{6}{7}
$$

#### 1.3 Smoothing
For smoothing, we have to write the Bayes rule for $X_0$ given $Y_1$:

$$
\pi_{X_0|Y_1}(x|y) \propto \pi_{Y_1|X_0}(y|x)\pi_{X_0}(x)
$$

Now the problem is the likelihood $\pi_{Y_1|X_0}(y|x)$ is not explicitly given. However, we can simply put the forward dynamics into the observation model, that is,

$$
Y_1 = X_1 + \sqrt{2}\Sigma_1 = \big(\frac{1}{2}X_0 + 1 + \Xi_0\big) + \sqrt{2}\Sigma_1
$$

Therefore we have

$$
Y_1 - 1 = \frac{1}{2}X_0 + \tilde{\Sigma}
$$

where $\tilde{\Sigma} = \Xi_0+ \sqrt{2}\Sigma_1 \sim N(0,3)$.
Now from the linear-Gaussian inference equation,

$$
\sigma_{x|y} = \frac{\sigma_x\sigma_y}{h^2\sigma_x+\sigma_y}
$$

$$
\pi_{X_0|Y_1}(x|y) \sim N\Big(\frac{h\sigma_{x|y}}{\sigma_y}y+\frac{\sigma_{x|y}}{\sigma_x}m, \sigma_{x|y}\Big)
$$

Substituting $m=-1$, $\sigma_x = 2$, $y=1$, $h = 1/2$ and $\sigma_y = 3$, we obtain the conditional variance

$$
\text{Var}\big[X_0 | Y_1=2\big] = \frac{2\cdot3}{2/4+3} = \frac{12}{7}
$$

and conditional mean

$$
\mathbb{E}\big[X_0 | Y_1 = 2\big] = \frac{2}{7}-\frac{6}{7} = -\frac{4}{7}
$$

#### 1.4
The smoothing step is computed from

$$
Y_1 = \alpha X_0 + \beta\Xi_0 + \sqrt{2}\Sigma_1
$$

and thus the cumulative noise variance is $\beta+2$.

Input: The dynamics parameter $\alpha$ and $\beta$, current mean $m$ and new observation $y$.

* m_1 = $\alpha\cdot m$   $\qquad\qquad$-- compute predictive mean
* v_1 = $\alpha^2 + \beta$ $\qquad\qquad$   -- compute predictive variance
* K = v_1/(v_1+2) $\qquad\qquad$   -- compute Kalman gain
* m_cond_1 = m_1 + K$\cdot$(y-m_1) $\qquad$ -- compute the conditional mean
* v_cond_1 = 2K  $\qquad\qquad\qquad$ -- compute the conditional variance
* K_sm = $\alpha$/($\alpha^2+\beta+2$) $\qquad$   -- compute Kalman gain for the smoothing step
* m_sm_0 = m + K_sm$\cdot$(y - $\alpha\cdot$ m) $\qquad$ -- compute the smoothed mean
* v_sm_0 = ($\beta$ + 2)$\cdot$ K_sm/$\alpha$  $\qquad\qquad$ -- compute the smoothed variance

Output: m_1, v_1, m_cond_1, v_cond_1, m_sm_0, v_sm_0

## Problem 2

In [11]:
"Particle simulation"
N = 1000
m = -1.
sigma_x0 = 2.
sigma_y = 2.
y = 2.

#Sample from prior and process noise
x0 = np.random.normal(loc = m ,scale=np.sqrt(sigma_x0),size=N)
xi = np.random.normal(size=N)

#Push forward
x1 = 0.5 * x0 + 1.0 + xi

#compute the likelihood weight
w_un = np.exp(-1/(2*sigma_y) * (2-x1)**2)
w = w_un / w_un.sum()

#compute the weighted mean and variance
m_cond = (w*x1).sum()
v_cond = (w*(x1-m_cond)*(x1-m_cond)).sum()

#compute the effective sample size
ess = 1/(w*w).sum()

#print results
print("Empirical mean is {:6.4f}. The theoretical value is {:6.4f}".format(m_cond,8/7))
print("Empirical variance is {:6.4f}. The theoretical value is {:6.4f}".format(v_cond,6/7))
print("The effecive sample size becomes {:4.2f}".format(ess))

Empirical mean is 1.1145. The theoretical value is 1.1429
Empirical variance is 0.8229. The theoretical value is 0.8571
The effecive sample size becomes 757.90


The result well agrees with the theoretical value. This implementation shall be a single time step in particle filtering.